# Retrieval Augmented Generation

RAG combines search (retrieval) with AI text generation. First retrieve relevant documents, then use them to generate informed responses.

![images/llm_2_rag_basic.png](images/llm_2_rag_basic.png)

## Connect to Weaviate

Connect to a Weaviate instance with third party credentials as needed.

In [1]:
# Refresh credentials & load the Weaviate IP
from helpers import update_creds

AWS_ACCESS_KEY, AWS_SECRET_KEY, AWS_SESSION_TOKEN = update_creds()

%store -r WEAVIATE_IP

In [2]:
import weaviate

client = weaviate.connect_to_local(
    WEAVIATE_IP,
    headers = {
        "X-AWS-Access-Key": AWS_ACCESS_KEY,
        "X-AWS-Secret-Key": AWS_SECRET_KEY,
        "X-AWS-Session-Token": AWS_SESSION_TOKEN,
    }
)

client.is_ready()

True

### Start with (R) - Retrieval

First step: retrieve relevant documents using search.

In [3]:
articles = client.collections.use("FinancialArticles")

response = articles.query.near_text(
    query="cloud revenue",
    limit=5,
    target_vector="title",
)

for item in response.objects:
    print(item.properties["article_title"])

IBM expects to exceed annual revenue target on resilient cloud momentum
It’s Been a Tough Run, but Don’t Dump Your Cloudflare Stock Just Yet
Alphabet CEO Sundar Pichai Says AI Start-Ups Are Choosing Google Cloud -- Time to Buy the Stock?
Splunk's Fiscal Q2 Revenue Declined: Here's Why That's Good News
Tech-sponsored study criticises plan to exclude non-EU cloud vendors


### Add (AG) - augmented generation - to make full RAG

Second step: use retrieved documents to generate informed responses.

#### Single Prompt

> Generate a response per **retrieved** object.

In [4]:
# Let's add some colour to our lives :)
BLUE   = "\033[94m"
PURPLE = "\033[95m"
RESET  = "\033[0"

In [5]:
from weaviate.classes.generate import GenerativeConfig

gen_config_aws = GenerativeConfig.aws(
    region="us-west-2",
    service="bedrock",
    model="us.amazon.nova-lite-v1:0"
)

response = articles.generate.near_text(
    query="cloud revenue",
    limit=5,
    target_vector="title",
    # STUDENT TODO
    # provide generative_provider and single_prompt parameters
    # `gen_config_aws`, and `Summarize this article to a sentence or two: {article}`
    # START_SOLUTION
    generative_provider=gen_config_aws,
    single_prompt="Summarize this article to a sentence or two: {article}"
    # END_SOLUTION
)

for item in response.objects:
    print(f"{BLUE}=== Source ===")
    print(item.properties["article_title"])

    print(f"{PURPLE}=== Generated Response ===")
    print(item.generative.text)
    print("\n")

=== Source ===
IBM expects to exceed annual revenue target on resilient cloud momentum
=== Generated Response ===
IBM reported better-than-expected quarterly revenue driven by strong demand for its digital services, despite a significant impact from the strong dollar, and now anticipates exceeding its full-year revenue growth targets.


=== Source ===
It’s Been a Tough Run, but Don’t Dump Your Cloudflare Stock Just Yet
=== Generated Response ===
Cloudflare (NYSE:NET) has faced recent declines in its stock price due to market corrections and competitors' disappointing outlooks, but its strong prospects and emphasis on security differentiate it from others in the Content Delivery Network market, making it a worthwhile hold for long-term investors.


=== Source ===
Alphabet CEO Sundar Pichai Says AI Start-Ups Are Choosing Google Cloud -- Time to Buy the Stock?
=== Generated Response ===
Alphabet's financial performance improved significantly in the second quarter of 2023, with Google Sear

#### Grouped Task

Generate one response using all retrieved documents together.

In [6]:
response = articles.generate.near_text(
    query="cloud revenue",
    limit=5,
    target_vector="title",
    generative_provider=gen_config_aws,
    # STUDENT TODO
    # provide grouped_task parameter
    # `Explain top 3 or so common trends or overall learnings in these articles. Please only use the provided content.`
    # START_SOLUTION
    grouped_task="Explain top 3 or so common trends or overall learnings in these articles. Please only use the provided content."
    # END_SOLUTION
)

print(f"{PURPLE}=== Generated Response ===")
print(response.generative.text)

print(f"{BLUE}=== Source ===")
for item in response.objects:
    print(item.properties["article_title"])

=== Generated Response ===
Here are the top three common trends or learnings from the provided articles:

1. **Cloud Services and Digital Transformation**:
   - **IBM**: IBM's focus on hybrid cloud services has driven robust growth, with cloud revenue rising by 11% and the company expecting to exceed its full-year revenue growth targets.
   - **Google Cloud**: Alphabet's Google Cloud is seeing significant traction, particularly with AI startups, and is becoming a profitable growth lever for the company.
   - **Splunk**: Splunk's transition to cloud services is accelerating, with cloud-based bookings and annual recurring revenue (ARR) growing strongly, despite short-term revenue declines.

2. **Impact of Economic Factors and Currency Fluctuations**:
   - **IBM**: The strong U.S. dollar has negatively impacted IBM's earnings, with a foreign exchange hit of $1.1 billion in the third quarter.
   - **Cloud Delivery Networks**: Companies like Cloudflare and Akamai are experiencing market vol

![images/llm_3_rag_weaviate.png](images/llm_3_rag_weaviate.png)

#### Specify which properties to use for grouped task

Control which document fields the AI uses for generation.

In [7]:
response = articles.generate.near_text(
    query="artificial intelligence",
    limit=50,
    target_vector="title",
    grouped_task="What are the top 3 most common types of articles, based on these titles? Please only use the provided content.",
    generative_provider=gen_config_aws,
    # STUDENT TODO
    # provide grouped_properties parameter
    # `["article_title"]`
    # START_SOLUTION
    grouped_properties=["article_title"]
    # END_SOLUTION
)

print(f"{PURPLE}=== Generated Response ===")
print(response.generative.text)

print(f"{BLUE}=== Source ===")
for item in response.objects:
    print(item.properties["article_title"])

=== Generated Response ===
Based on the provided article titles, the top 3 most common types of articles are:

1. Investment and stock market analysis: Many articles focus on specific stocks, investment strategies, and market trends related to AI and technology companies. Examples include "The 3 Best Growth Stocks to Buy in the Artificial Intelligence Sector," "Buy Alert: 3 AI Stocks Nearing Super Attractive Entry Points," and "Investors Should Buy Nvidia Stock."

2. Company and product news: Several articles discuss recent developments, product launches, and financial performance of companies in the AI and technology sectors. Examples include "Nvidia Dominates AI Silicon, But That Doesn't Justify A $1 Trillion Valuation," "Alphabet CEO Sundar Pichai Says AI Start-Ups Are Choosing Google Cloud -- Time to Buy the Stock?," and "Apple Unveils Vision Pro Headset, IOS 17, 15-inch MacBook Air And More."

3. Opinions and expert analysis: Some articles feature opinions and insights from expert

## Set default Generative model

Configure a default model to avoid specifying it each time.

In [8]:
from weaviate.classes.config import Reconfigure

articles.config.update(
    generative_config=Reconfigure.Generative.aws(
        region="us-west-2",
        service="bedrock",
        model="us.amazon.nova-pro-v1:0"
    )
)

Try generative query without specifying the model (uses default).

In [9]:
response = articles.generate.near_text(
    query="cryptocurrency regulation",
    limit=5,
    target_vector="title",
    grouped_task="Briefly, what topics are covered in these articles? Please only use the provided content.",
)

print(f"{PURPLE}=== Generated Response ===")
print(response.generative.text)

print(f"{BLUE}=== Source ===")
for item in response.objects:
    print(item.properties["article_title"])

=== Generated Response ===
1. **US consumer watchdog proposes rules for Big Tech payments, digital wallets**: The article discusses the Consumer Financial Protection Bureau's proposal to regulate tech giants' digital payments and smartphone wallet services, subjecting companies like Alphabet, Apple, and PayPal to bank-like supervision.

2. **Is Cryptocurrency Really an Investment for the Long Term?**: The article explores the viability of cryptocurrency as a long-term investment, distinguishing between digital currencies (like Bitcoin) and tokens (like Theta Fuel), and discussing the factors investors should consider for each.

3. **PayPal Unveils Crypto Integration**: The article reports on PayPal's launch of the Cryptocurrencies Hub, allowing users to engage with Bitcoin and other cryptocurrencies within their PayPal accounts, and discusses the implications and reactions within the crypto community.

4. **PayPal’s Recent Crypto Moves Won’t be Enough to Resist Current Headwinds**: The

## Close the client

Always close your connection when finished.

In [10]:
client.close()